### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2020 ###

# Grade Boosters #

#### Zhiwen Xu zxu365@wisc.edu, Peter Swanson pswanson2@wisc.edu, Donald Gu lgu33@wisc.edu

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
2. [Mathematical Model](#2.-Mathematical-model)
3. [Solution](#3.-Solution)
4. [Results and Discussion](#4.-Results-and-Discussion)
5. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

This project develops an algorithm to help students in the University of Wisconsin-Madison Computer Sciences department develop an optimal schedule based on their interests. Users of the algorithm will be able to find a course list that satisfies all departmental degree requirements while maximizing the average GPA earned in each class, and they also have the option to find a  course list that satisfies all departmental degree requirements while maximizing the number of classes that focus on an area of Computer Sciences in which they are particularly interested. For example, a student can choose to focus on information security during their time in the Computer Sciences program, and the algorithm will give them a schedule with the maximum possible number of courses with that focus while still satisfying all degree requirements.  

The following is a list of the areas of Computer Sciences a student can choose from:  

| Abbreviation | Area Name |
|-------------|-------------|
|Ai | Artificial Intelligence | 
|Ar | Computer Architecture | 
|Sy | Computer Systems | 
|Ap | Mobile Application Development |  
|Th | Theory and Algorithms |  
|Ec | Economics |  
|Ro | Robotics |  
|Da | Data Science |  
|Bi | Biomedical |  
|Is | Information Security |  


The need for such a tool has been present for a long time. Every semester around course enrollment time, students spend hours searching each class, and trying to decide if they are interested in taking a class, and whether it will fit in their schedules. Some students are only interested in getting into the easiest classes, while others have specific interests that they would like to spend all of their class time learning about. 

Many students use the website [madgrades.com](madgrades.com) to look at the grade distribution of previous semesters of courses they want to take, and the dataset used by that website is all online in a relational database. Using this database, we wrote queries to extract every course that could fulfill any requirement for the Computer Sciences major, along with their corresponding average GPAs. In order to determine the prerequisites of each course, we manually curated a graph structure by reading off the [Computer Sciences webpage](https://guide.wisc.edu/undergraduate/letters-science/computer-sciences/computer-sciences-bs/#requirementstext). In order to determine which area of computer sciences a particular class focuses on, we read the course description and filled in a graph structure based on that information.

The algorithm takes into account course prerequisites, students' areas of interest within computer science, and course difficulty (Average GPA). Furthermore, this project explores the trade-off between areas of interests and average GPA to find each student's optimal schedule. 

The purposes of this project are to:  
1. look for an algorithm to best serve the students as they go through the computer sciences program  
2. give advisors of the Computer Sciences program information to better serve their undergraduate students  
3. provide social scientists and other social, administrative, and political workers insights of computer science trends within American Higher Education. 

In this report, we will develop two models: the first will find the easiest path to degree, while the second will determine the optimal schedule given a specific area of interest. In the results section, we will list some of the schedules we found, given  different areas of interest. See the conclusion for a summary of our findings as well as possible future directions.

## 2. Mathematical model

Model 1 and Model 2 share the same parameters, decision variables, and constraints. The difference between the two lies in the objective.

### 2.1. Parameters  ###

#### 2.1.1 Courses ####

73 courses are used in our model; most are computer science courses, and the rest are mathematics, statistics, or ECE courses. They are represented by an array named $courses$.

#### 2.1.2 GPA#### 

We obtained a dataset from Kaggle that contains every class offered at UW-Madison with their corresponding number of As, ABs, Bs, Cs, Ds, and Fs given for each semester from 2006 to 2017. The dataset can be downloaded at https://www.kaggle.com/Madgrades/uw-madison-courses#grade_distributions.csv. GPA is calculated from this dataset and is stored in courses_and_GPA.csv. 

GPA is represented by an array named $gpa$. $gpa_i$ is the corresponding GPA for course i. 

#### 2.1.3 Prerequisites & Graduation Requirement List ####

We obtained the pre-requisites and graduation requirements from UW-Madison CS major requirement webpage (https://guide.wisc.edu/undergraduate/letters-science/computer-sciences/computer-sciences-bs/#requirementstext). 

Let the below variables be arrays that include the index of courses that satisfy different graduation requirements:

$ basic\_cs $: basic computer sciences $$ $$
    $ basic\_math $: basic mathematics $$ $$
    $ add\_math $: additional mathematics (beyond calculus) $$ $$
    $ theory $: theory of computer science $$ $$
    $ soft\_hard $: software and hardware $$ $$
    $ app $: application $$ $$
    $ elect $: elective $$ $$



Let array $pred$ records the pre-requisites for each course; $pred_i$ represents the indexes of pre-requisites courses for course i.  

#### 2.1.4 Number of Courses ####

NUM_COURSES refers to at least how many courses to take to satisfy degree requirement. It is set to be 15. 

#### 2.1.4 Course Topics ####

For each course topic, an array containing indexes of courses is created. The topics are coded in the following arrays:

$Ai$: Artificial Intelligence

$Ar$: Computer Architecture

$Sy$: Computer Systems

$Ap$: Mobile Application Development

$Th$: Theory and Algorithms

$Ec$: Economics

$Ro$: Robotics

$Da$: Data Science

$Bi$: Biomedical

$Is$: Information Security


### 2.2. Decision Variables ###

Let $i$ be the index of possible classes listed from the class table, $x_i$ be a binary variable for each class. The length of $x$ is 73, and $x_i=1$ when the class is taken, $x_i=0$ when the class is not taken.

### 2.3. Constraints ###

1) Graduation Constraint: For each requirement category, specific number of courses need to be taken:

$$\quad \sum_{i} x_i = 5 \ for \ i \in \ basic\_cs $$
    $$\quad \sum_{i} x_i = 2 \ for \ i \in \ basic\_math,$$
    $$\quad \sum_{i} x_i \geq 2 \ for \ i \in \ add\_math ,$$
    $$\quad \sum_{i} x_i \geq 1 \ for \ i  \in \ theory ,$$
    $$\quad \sum_{i} x_i \geq 2 \ for \ i \in \ soft\_hard ,$$
    $$\quad \sum_{i} x_i \geq 1 \ for \ i \in \ app ,$$
    $$\quad \sum_{i} x_i \geq 2 \ for \ i \in \ elect .$$
    
For example, in order to graduate, a student must take at least one class in application area.

2) Prerequisite Constraint: For each class i, its pre-requisites must be taken at first:

$$ x_j >= x_i \ for \ j \in \ pred_i $$

3) A class can not count for more than one degree requirement:

$$\quad \sum_{i} x_i = \ NUM\_COURSES $$

### 2.4. Objective ###

#### Model 1  

Maximize overall GPA:

$$max(\quad \sum_{i = 1}^{73}gpa_i*x_i)$$

#### Model 2  

Maximize the number of courses in interested topic:

$$max(\quad \sum_{i \in topic}x_i)$$ where topic can be any of the course topics. In our implementation, we chose topic $Ai$ as an example.

## 3. Solution ##  
### DATA

In [1]:
### Run once to import packages (if needed)
using Pkg
Pkg.add("CSV")
Pkg.add("Cbc")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  99.9 %0.0 %=======>                                 ]  16.2 %]  32.2 %>               ]  60.7 %===============================>         ]  76.8 % Resolving package versions...
 Installed StaticArrays ─ v0.12.3
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
  [90137ffa] ↑ StaticArrays v0.12.2 ⇒ v0.12.3
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


In [2]:
# SETUP DATA
using CSV

raw = CSV.read("courses_and_GPA.csv")
(m,n) = size(raw)

courses = raw[:,1]
gpa = convert(Matrix{Float64}, raw[:,2:n])
pre = [[],[],[],[2],[2,3],[3],[3],[69,70,3],[51],[51],[49],[51],[6,7],[55,3],[51,59,3],[70,6],[3,51],[51],[18,3],[69,1],[45,51],[3,69],[5,6],[5,6],[5,6],[1],[3,68],[23],[47,3,5],[4,5],[6,47],[69,6],[5,6],[55,73],[1],[3,69],[70,6],[],[3,51],[],[24],[24],[32],[],[44],[44],[69],[47],[71,73],[69],[69],[58],[52],[47],[69],[47],[],[57],[47],[47],[51],[47],[53],[56,57],[64],[64],[56],[],[],[],[47],[48],[68]]                                 

basic_cs = [2,3,5,6,70]
basic_math = [68,69]
add_math = [55,73,8,10,14,15,18,19,46,47,50,51,52,53,54,56,58,60,61,62,12,63,64,65,66,67,48,49,71,72]
theory = [37,16]
soft_hard = [7,13,23,24,30,33,41,42]
app = [8,9,14,15,17,18,22,27,28,29,32,35,]
elect = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,40]
len = length(gpa)

# Data for course topics
Ai = [8, 11, 14, 15, 16, 17, 18, 19, 20, 21, 26, 27, 28, 40, 45, 46, 55, 56, 61]
Ar = [2, 4, 5, 24, 25, 30, 40, 44, 45]
Sy = [2, 4, 5, 6, 7, 13, 20, 23, 24, 25, 29, 30, 33, 38, 40, 41, 42, 43, 44, 45]
Ap = [1, 3, 6, 7, 13, 17, 20, 21, 22, 27, 32, 33, 35, 38, 39, 40, 41, 42, 43]
Th = [3, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 25, 26, 27, 28, 31, 37, 40, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 70, 71, 72, 73]
Ec = [10, 11, 17, 27, 40, 46, 48, 49, 60, 71, 72]
Ro = [4, 20, 21, 27, 40, 44, 45, 46, 73]
Da = [9, 11, 14, 17, 18, 19, 27, 33, 34, 40, 46, 48, 49, 55, 56, 60, 61, 71, 72, 73]
Bi = [34, 36, 40]
Is = [9, 12, 32, 40, 41];

## Model 1 - Maximize GPA

In [3]:
NUM_COURSES = 15

using JuMP, Cbc

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# whether or not a course is taken
@variable(m, x[1:len], Bin)

# Must satisfy degree requirements
@constraint(m, sum( x[basic_cs[i]] for i in 1:length(basic_cs) ) == 5)
@constraint(m, sum( x[basic_math[i]] for i in 1:length(basic_math) ) == 2)
@constraint(m, sum( x[add_math[i]] for i in 1:length(add_math) ) >= 2)
@constraint(m, sum( x[theory[i]] for i in 1:length(theory) ) >= 1)
@constraint(m, sum( x[soft_hard[i]] for i in 1:length(soft_hard) ) >= 2)
@constraint(m, sum( x[app[i]] for i in 1:length(app) ) >= 1)
@constraint(m, sum( x[elect[i]] for i in 1:length(elect) ) >= 2)

# A class can not count for more than one degree requirement
@constraint(m, sum( x[i] for i in 1:len ) == NUM_COURSES)

# Must satisfy prerequisite requirements
for i in 1:len
    for j in pre[i]
        @constraint(m, x[i] <= x[j])
    end
end

# Maximize GPA
@objective(m, Max, sum(x[i]*gpa[i] for i in 1:len) / NUM_COURSES);

optimize!(m)

println("Optimal courselist: ")
println()
opt_gpa = 0
m1_courses = zeros(Int64, NUM_COURSES)
j = 1
for i in 1:len
    if (value(x[i]) > 0)
        m1_courses[j] = i
        j += 1
        println(courses[i])
        opt_gpa += gpa[i]
    end
end
opt_gpa /= NUM_COURSES
println()
println()
println("The optimal GPA will be ",opt_gpa)

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1273


Optimal courselist: 

COMP SCI 252
COMP SCI 300
COMP SCI 354
COMP SCI 400
COMP SCI 407
COMP SCI 506
COMP SCI 547
COMP SCI 558
COMP SCI 577
COMP SCI 579
MATH 234
MATH 461
MATH 221
MATH 222
MATH 240


The optimal GPA will be 3.2599699242


## Model 2 - Maximize Desired Courses

In [4]:
# PERFORM CALCULATION
NUM_COURSES = 15

using JuMP, Cbc

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# whether or not a course is taken
@variable(m, x[1:len], Bin)

# Must satisfy degree requirements
@constraint(m, sum( x[basic_cs[i]] for i in 1:length(basic_cs) ) == 5)
@constraint(m, sum( x[basic_math[i]] for i in 1:length(basic_math) ) == 2)
@constraint(m, sum( x[add_math[i]] for i in 1:length(add_math) ) >= 2)
@constraint(m, sum( x[theory[i]] for i in 1:length(theory) ) >= 1)
@constraint(m, sum( x[soft_hard[i]] for i in 1:length(soft_hard) ) >= 2)
@constraint(m, sum( x[app[i]] for i in 1:length(app) ) >= 1)
@constraint(m, sum( x[elect[i]] for i in 1:length(elect) ) >= 2)

# A class can not count for more than one degree requirement
@constraint(m, sum( x[i] for i in 1:len ) == NUM_COURSES)

# Must satisfy prerequisite requirements
for i in 1:len
    for j in pre[i]
        @constraint(m, x[i] <= x[j])
    end
end

# Maximize Desired Courseload
# Change Ai to any of the other desired areas of interest
# To find another optimal schedule
@objective(m, Max, sum(x[i] for i in Ai));

optimize!(m)

println("Optimal courselist: ")
println()
opt_gpa = 0
m2_courses = zeros(Int64, NUM_COURSES)
j = 1
for i in 1:len
    if (value(x[i]) > 0)
        m2_courses[j] = i
        j += 1
        println(courses[i])
        opt_gpa += gpa[i]
    end
end
opt_gpa /= NUM_COURSES
println()
println()
println("The optimal GPA will be ",opt_gpa)

Optimal courselist: 

COMP SCI 252
COMP SCI 300
COMP SCI 354
COMP SCI 400
COMP SCI 412
COMP SCI 520
COMP SCI 536
COMP SCI 540
COMP SCI 545
COMP SCI 564
COMP SCI 639
MATH 340
MATH 221
MATH 222
MATH 240


The optimal GPA will be 3.1451743004666666


### Setup some helper methods

In [5]:
# Check if course c is added to schedule s
function isAdded(s,c)
    for i in 1:8
        for j in 1:2
            if (s[i,j] == c)
                return true
            end
        end
    end
    return false
end

# Check if the prerequisites to course c have been added to schedule s
function preSat(s,c,j,k)
    for i in pre[c]
        if (!isAdded(s,i))
            return false
        end
        if (k == 2 && s[j,1] == i)
            return false
        end
    end
    return true
end
;

### Print a possible schedule for model 1

In [6]:
# Set up some variables
sched = zeros(Int64,8,2)
sched[8,2] = -1
j = 1
k = 1

# Populate the schedule
while (isAdded(sched,0)) 
    for i in m1_courses
        if (!isAdded(sched,i) && preSat(sched,i,j,k))
            sched[j,k] = i
            if (k == 2)
                j += 1
                k = 1
            else
                k += 1
            end
            break
        end
    end
end

# Print the schedule
for i in 1:8
    println("Starting semester ", i)
    println("====================")
    for j in 1:2
        if (i == 8 && j == 2)
            break
        end
        println(courses[sched[i,j]])
    end
    println("\n")
end

Starting semester 1
COMP SCI 252
COMP SCI 300


Starting semester 2
COMP SCI 354
COMP SCI 400


Starting semester 3
COMP SCI 407
COMP SCI 579


Starting semester 4
COMP SCI 506
MATH 221


Starting semester 5
MATH 222
MATH 240


Starting semester 6
COMP SCI 577
MATH 234


Starting semester 7
COMP SCI 547
COMP SCI 558


Starting semester 8
MATH 461




### Print a possible schedule for model 2

In [7]:
# Set up some variables
sched = zeros(Int64,8,2)
sched[8,2] = -1
j = 4
k = 2
iterations = 0
m47 = 0

# Take the basic_cs and basic_calc courses first
sched[1,1] = basic_math[1]
sched[1,2] = basic_cs[1]
sched[2,1] = basic_math[2]
sched[2,2] = basic_cs[2]
sched[3,1] = basic_cs[3]
sched[3,2] = basic_cs[4]
sched[4,1] = basic_cs[5]
for i in m2_courses
    if (i == 47) # Math 234
        sched[4,2] = 47
        j = 5
        k = 1
    end
end
for i in m2_courses
    if (m47 == 0 && i == 51) # Math 234
        sched[4,2] = 51
        j = 5
        k = 1
    end
end


# Populate the schedule
while (isAdded(sched,0) && iterations < 10000) 
    for i in m2_courses
        if (!isAdded(sched,i) && preSat(sched,i,j,k))
            sched[j,k] = i
            if (k == 2)
                j += 1
                k = 1
            else
                k += 1
            end
            break
        end
        iterations += 1
    end
end

# Print the schedule
for i in 1:8
    println("Starting semester ", i)
    println("====================")
    for j in 1:2
        if (i == 8 && j == 2)
            break
        end
        println(courses[sched[i,j]])
    end
    println("\n")
end


Starting semester 1
MATH 221
COMP SCI 252


Starting semester 2
MATH 222
COMP SCI 300


Starting semester 3
COMP SCI 354
COMP SCI 400


Starting semester 4
MATH 240
COMP SCI 536


Starting semester 5
COMP SCI 412
COMP SCI 520


Starting semester 6
COMP SCI 540
COMP SCI 545


Starting semester 7
COMP SCI 564
COMP SCI 639


Starting semester 8
MATH 340




## 4. Results and Discussion ##

### 4.1 Results

The course schedule to maximize GPA is: 

COMP SCI 252, COMP SCI 300, COMP SCI 354, COMP SCI 400, COMP SCI 407, COMP SCI 506, COMP SCI 547, COMP SCI 558, COMP SCI 577, COMP SCI 579, MATH 234, MATH 461, MATH 221, MATH 222, MATH 240

With an average GPA of **3.26**

Using the prerequisite dependency relationships, a possibe 8-semester schedule for these courses is:

| Semester | Course 1 | Course 2 |
|-------------|-------------|-----|
| 1 | COMP SCI 252 | COMP SCI 300 |
| 2 | COMP SCI 354 | COMP SCI 400 |
| 3 | COMP SCI 407 | COMP SCI 579 |
| 4 | COMP SCI 506 | MATH 221 |
| 5 | MATH 222 | MATH 240 |
| 6 | COMP SCI 577 | MATH 234 |
| 7 | COMP SCI 547 | COMP SCI 558 |
| 8 | MATH 461 |  |

In our second model, we calculated the optimal schedule based on a student's chosen area of interest. By changing the prefered area of interest in the code block in model 2, the model will find ten different optimal schedules based on the ten different areas of interest. We have chosen three schedules to list here as an example, but to see a different schedule, simply change the chosen array in the objective function in model 2, run the code, and then run the code block titled "Print a possible schedule for model 2". Note that this will give one of many possible schedules. The following schedules are not the only order in which to take the courses.

#### Artificial Intelligence possible schedule:  
| Semester | Course 1 | Course 2 |
|-------------|-------------|-----|
| 1 | COMP SCI 252 | COMP SCI 300 |
| 2 | COMP SCI 354 | COMP SCI 400 |
| 3 | COMP SCI 536 | COMP SCI 545 |
| 4 | COMP SCI 564 | COMP SCI 639 |
| 5 | MATH 221 | COMP SCI 540 |
| 6 | MATH 222 | MATH 340 |
| 7 | MATH 240 | COMP SCI 412 |
| 8 | COMP SCI 520 |  |  

#### Data Science possible schedule: 
| Semester | Course 1 | Course 2 |
|-------------|-------------|-----|
| 1 | COMP SCI 252 | COMP SCI 300 |
| 2 | COMP SCI 354 | COMP SCI 400 |
| 3 | COMP SCI 407 | COMP SCI 564 |
| 4 | COMP SCI 639 | MATH 221 |
| 5 | COMP SCI 540 | MATH 222 |
| 6 | MATH 340 | MATH 240 |
| 7 | COMP SCI 577 | STAT 324 |
| 8 | COMP SCI 567 |  |  

#### Computer Systems possible schedule:
| Semester | Course 1 | Course 2 |
|-------------|-------------|-----|
| 1 | COMP SCI 252 | COMP SCI 300 |
| 2 | COMP SCI 352 | COMP SCI 354 |
| 3 | COMP SCI 400 | COMP SCI 552 |
| 4 | COMP SCI 564 | COMP SCI 579 |
| 5 | COMP SCI 639 | MATH 221 |
| 6 | MATH 222 | MATH 240 |
| 7 | COMP SCI 412 | COMP SCI 577 |
| 8 | MATH 319 |  |  

### 4.2 Discussion  

Based on pre-requisites, graduation requirements, and grade distributions, model 1 gives a Computer Sciences student the easiest course load to graduate. With the highest possible GPA in CS courses, the student may find it easier to look for jobs or apply for graduate programs. Avoiding the challenging courses, the student will also face less pressure while studying. 

However, there are some limitations in model 1. For example, course credit is not reflected in the GPA calculation as each course is weighted equally, and therefore, the optimal GPA is not precise. However, if course credit is included, the model will turn into a complicated non-linear programming model, which can be hard to solve. Furthermore, the calculated optimal GPA will not lose too much precision since most courses have three or four credits. Therefore, course credit is left out of the model. A major limitation of this model is that some courses in the generated list may not be interesting to every student, which caused us to create model 2, which accounts for student interests.

Model 2 enables a student to take desired courses as much as possbile, without considering GPA. It is possible that the coureses the student is interested in will harm their GPA on average. On the other hand, it is possible that they will do well in those classes, since the student is interested in them and thus may work hard or already know about the course material. The student can also compare the outputs from both models and make a balanced decision between the two.

We further refined the output by creating a potential course schedule. The schedule is built upon considerations on semester load and prerequistes. Offered a schedule, a student can have a clearer idea about when to take which course. 

Nonetheless, there are some improvements to be made for our models. Deciding which course to take is a complex process and usually a student also takes many other factors into consideration, such as professor rating, course load and semester dependent course offerings. It is complicated to combine all these important factors into our model since in practical setting, different professors teach at different semesters, and they also give different grade distributions. In future development, these factors can be considered and the tradeoff between different factors can be explored as well.

## 5. Conclusion ##
We have successfully found an algorithm and solved for the minimal number of courses with maximal GPA needed to graduate from University of Wisconsin Madison Department of Computer Sciences. We have concluded that the results for this specific optimization problem are accurate. However, there are still limitations on GPA prediction that lead to inaccuracy. Factors including the difficulties of each class, the credit load of each class, the number of students in each class, and standard deviation of total grade points are important in determining an individual GPA for a specific course. In order to take these factors into account, the model would require more data sets related to the aforementioned factors and constraints. 

Furthermore, we found an algorithm to create schedules with pre-determined sets of preferred areas of interests. This algorithm is useful for administrators and school advisors to provide students with feedback and match their interests with a preferred course schedule. This type of advising software is usually missing in many computer science undergraduate programs, and administrators who seek to provide students with online advising could potentially use this algorithm. However, the preferred courses data should be reviewed by a faculty board and the software should be more thoroughly tested before being implemented in any large-scale applications. It is recognized that many factors are out of the scope of this existing algorithm, and this algorithm could be further improved by increasing constraints and comparing their trade-offs against each other.

Finally, we also intended for this algorithm to provide insights into trends of students' interests and course selections for administration boards. This algorithm alone is not able to provide the results to analyze trends among students; however, one can gain more insights by gathering data from the students who utilize software implemented by this algorithm, such as their satisfaction rate, and if they ended up following the advice from our algorithm.

## 6. Author Contributions

Student A = Zhiwen  
Student B = Peter  
Student C = Donald  

#### 1. Modelling  
Student A: Percentage contribution 33%  
Student B: Percentage contribution 33%  
Student C: Percentage contribution 33%  

  
#### 2. Analysis  
Student A: Percentage contribution 33%  
Student B: Percentage contribution 33%  
Student C: Percentage contribution 33%  


#### 3. Data Gathering  
Student A: Percentage contribution 33%  
Student B: Percentage contribution 33%  
Student C: Percentage contribution 33%  


#### 4. Software Implementation  
Student A: Percentage contribution 33%  
Student B: Percentage contribution 33%  
Student C: Percentage contribution 33%  


#### 5. Report writing and poster presentation   
Student A: Percentage contribution 33%  
Student B: Percentage contribution 33%  
Student C: Percentage contribution 33%  